In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100
SUMMARY_DIR = './mnist'

MNIST = input_data.read_data_sets("./MNIST_data",one_hot = True)

Extracting ./MNIST_data\train-images-idx3-ubyte.gz
Extracting ./MNIST_data\train-labels-idx1-ubyte.gz
Extracting ./MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data\t10k-labels-idx1-ubyte.gz


In [4]:
import numpy
def get_train_batch(batch_size, fake_data=False):

    a = batch_size/128
    b = batch_size%128
    
    #shuffle the data 
   #perm = numpy.arange(self._num_examples) 
   #numpy.random.shuffle(perm) 
        
    for i in range(a):
        batch_xs, batch_ys =MNIST.train.next_batch(128)
    
    for i in range(b):
        batch_xs, batch_ys =MNIST.train.next_batch(b)

    

In [5]:
with tf.name_scope('input') as scope:
    X= tf.placeholder(tf.float32, [None, 784], name = 'image')
    y= tf.placeholder(tf.float32, [None, 10], name = 'label')

In [6]:
with tf.variable_scope('layer1') as scope:
    W1 = tf.get_variable("W", shape = [784,512], initializer = tf.contrib.layers.xavier_initializer())
   # W1 = np.random.randn(784, 512)/ np.sqrt(784)
    b1 = tf.Variable(tf.random_normal([512]))
    L1 = tf.nn.relu(tf.add(tf.matmul(X, W1),b1))
    
    tf.summary.histogram("X",X)
    tf.summary.histogram("weights", W1)
    tf.summary.histogram("bias", b1)
    tf.summary.histogram("layer",L1)

In [7]:
with tf.variable_scope('layer2') as scope:
    W2 = tf.get_variable("W", shape = [512,512], initializer = tf.contrib.layers.xavier_initializer())
   # W1 = np.random.randn(784, 512)/ np.sqrt(784)
    b2 = tf.Variable(tf.random_normal([512]))
    L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2),b2))
    
    
    #tf.summary.histogram("weights", W1)
    tf.summary.histogram("weights", W2)
    tf.summary.histogram("bias", b2)
    tf.summary.histogram("layer",L2)

In [8]:
with tf.variable_scope('layer3') as scope:
    W3 = tf.get_variable("W", shape=[512, 10], initializer = tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([10]))
    y_ = tf.add(tf.matmul(L2, W3), b3)
    
    tf.summary.histogram("weights", W3)
    tf.summary.histogram("bias", b3)
    tf.summary.histogram("logits", y_)

In [ ]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_, labels = y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)
tf.summary.scalar("loss", loss)

summary = tf.summary.merge_all()

global_step =0

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(SUMMARY_DIR, sess.graph)
    for epoch in range(training_epochs):
        total_batch = int(MNIST.train.num_examples / batch_size)
        avg_loss =0
        
        for i in range(total_batch):
            batch_xs, batch_ys =MNIST.train.next_batch(batch_size)
            #get_train_batch(batch_size)
            feed_dict = {X: batch_xs, y: batch_ys}
            s, l,  _ = sess.run([summary, loss, optimizer], feed_dict = feed_dict)
            writer.add_summary(s, global_step=global_step)
            global_step +=1
            avg_loss+= l
        print('Epoch:','%02d'%(epoch + 1), 'loss=','{:.6f}'.format(avg_loss/total_batch))
        
    correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    acc = sess.run(accuracy,feed_dict = {X:MNIST.test.images, y: MNIST.test.labels})
    print('Test accuracy!:', acc)
    